In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

%pylab inline
import seaborn as sns

path='./data/data137263/'

Populating the interactive namespace from numpy and matplotlib


In [2]:
train=pd.read_csv(path+'pubg_train.csv.zip')
test=pd.read_csv(path+'pubg_test.csv.zip')

train['type']='trian'
test['type']='test'
data=pd.concat([train,test])

In [3]:
tmpcols=['kill_distance_x_min','kill_distance_x_max', 'kill_distance_y_min', 'kill_distance_y_max']
data[tmpcols]=data[tmpcols]+1
data[tmpcols]=data[tmpcols].fillna(0)

In [4]:
process_tmp=data[['match_id','team_id']]
process_cols=['player_assists','player_dbno','player_dist_ride','player_dist_walk',
'player_dmg','player_kills','kill_distance_x_min','kill_distance_x_max','kill_distance_y_min','kill_distance_y_max']

tmp=data.groupby(['match_id','team_id'])[process_cols].min().reset_index()
# tmp.columns = ['match_id','team_id']+[i+'_min' for i in process_cols]
process_tmp=process_tmp.merge(tmp,on=['match_id','team_id'],how='left')

tmp=data.groupby(['match_id','team_id'])[process_cols].max().reset_index()
# tmp.columns = ['match_id','team_id']+[i+'_max' for i in process_cols]
process_tmp=process_tmp.merge(tmp,on=['match_id','team_id'],how='left')

tmp=data.groupby(['match_id','team_id'])[process_cols].sum().reset_index()
# tmp.columns = ['match_id','team_id']+[i+'_sum' for i in process_cols]
process_tmp=process_tmp.merge(tmp,on=['match_id','team_id'],how='left')

# tmp=data.groupby(['match_id','team_id'])[process_cols].mean().reset_index()
# tmp.columns = [i+'_mean' for i in tmp.columns]
# tmp.rename(columns={'match_id_mean':'match_id','team_id_mean':'team_id'},inplace=True)
# process_tmp=process_tmp.merge(tmp,on=['match_id','team_id'],how='left')

process_tmp=process_tmp.drop_duplicates()
data=data[['match_id','team_id','game_size','party_size','team_placement','type']]
data=data.drop_duplicates()
data=data.merge(process_tmp,on=['match_id','team_id'],how='left')

train=data[data['type']=='trian']
test=data[data['type']=='test']

In [5]:
lgb_cols=['match_id', 'team_id', 'game_size', 'party_size', 
       'player_assists_x', 'player_dbno_x', 'player_dist_ride_x',
       'player_dist_walk_x', 'player_dmg_x', 'player_kills_x',
       'kill_distance_x_min_x', 'kill_distance_x_max_x',
       'kill_distance_y_min_x', 'kill_distance_y_max_x', 'player_assists_y',
       'player_dbno_y', 'player_dist_ride_y', 'player_dist_walk_y',
       'player_dmg_y', 'player_kills_y', 'kill_distance_x_min_y',
       'kill_distance_x_max_y', 'kill_distance_y_min_y',
       'kill_distance_y_max_y', 'player_assists', 'player_dbno',
       'player_dist_ride', 'player_dist_walk', 'player_dmg', 'player_kills',
       'kill_distance_x_min', 'kill_distance_x_max', 'kill_distance_y_min',
       'kill_distance_y_max']

In [6]:
train['count']=train.groupby('match_id')['team_placement'].transform('count')
test['count']=test.groupby('match_id')['team_placement'].transform('count')

# tmp=train[['match_id','count']]
# tmp=tmp.drop_duplicates(keep='first')
# g_train=tmp['count'].values.tolist()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
from sklearn.metrics import mean_absolute_error

k_fold=3
sub_preds = np.zeros(test.shape[0])
for i in range(3):
    valid_idx=train[i*50000:(i+1)*50000]
    train_idx=train[~train['match_id'].isin(valid_idx['match_id'].tolist())]
    
    # 训练集与验证集的用户分组
    g_train = train_idx.groupby(['match_id'], as_index=False).count()["team_placement"].values
    
    g_val = valid_idx.groupby(['match_id'], as_index=False).count()["team_placement"].values
    
    # 定义模型
    lgb_ranker = lgb.LGBMRanker(boosting_type='gbdt', num_leaves=31, reg_alpha=0.0, reg_lambda=1,objective='regression',
                            max_depth=-1, n_estimators=3000, subsample=0.7, colsample_bytree=0.7, subsample_freq=1,
                            learning_rate=0.01, min_child_weight=50, random_state=2022, n_jobs= 16)
    # 训练模型
    lgb_ranker.fit(train_idx[lgb_cols], train_idx['team_placement'], group=g_train,
                   eval_set=[(valid_idx[lgb_cols], valid_idx['team_placement'])], eval_group= [g_val])
    
    # 预测验证集结果
    valid_idx['pred_score'] = lgb_ranker.predict(valid_idx[lgb_cols])
    
    print(100 - mean_absolute_error(valid_idx['team_placement'].tolist(), valid_idx['pred_score'].tolist()))

    sub_preds += lgb_ranker.predict(test[lgb_cols], lgb_ranker.best_iteration_)
    
# 测试集的预测结果，多次交叉验证求平均,将预测的score和对应的rank特征保存，可以用于后面的staking，这里还可以构造其他更多的特征
test['pred_score'] = sub_preds / k_fold

# test['pred_rank'] = test.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')

[1]	valid_0's l2: 368.9
[2]	valid_0's l2: 362.64
[3]	valid_0's l2: 356.184
[4]	valid_0's l2: 349.843
[5]	valid_0's l2: 343.619
[6]	valid_0's l2: 337.671
[7]	valid_0's l2: 331.711
[8]	valid_0's l2: 325.964
[9]	valid_0's l2: 320.216
[10]	valid_0's l2: 314.59
[11]	valid_0's l2: 309.076
[12]	valid_0's l2: 303.665
[13]	valid_0's l2: 298.366
[14]	valid_0's l2: 293.166
[15]	valid_0's l2: 288.052
[16]	valid_0's l2: 284.12
[17]	valid_0's l2: 279.197
[18]	valid_0's l2: 274.351
[19]	valid_0's l2: 269.617
[20]	valid_0's l2: 264.985
[21]	valid_0's l2: 260.438
[22]	valid_0's l2: 255.977
[23]	valid_0's l2: 251.604
[24]	valid_0's l2: 247.317
[25]	valid_0's l2: 243.095
[26]	valid_0's l2: 238.983
[27]	valid_0's l2: 234.936
[28]	valid_0's l2: 230.965
[29]	valid_0's l2: 227.086
[30]	valid_0's l2: 223.275
[31]	valid_0's l2: 219.641
[32]	valid_0's l2: 215.974
[33]	valid_0's l2: 212.367
[34]	valid_0's l2: 208.846
[35]	valid_0's l2: 205.45
[36]	valid_0's l2: 202.076
[37]	valid_0's l2: 198.94
[38]	valid_0's l2

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


96.15426522679942
[1]	valid_0's l2: 401.548
[2]	valid_0's l2: 394.686
[3]	valid_0's l2: 387.654
[4]	valid_0's l2: 380.747
[5]	valid_0's l2: 373.971
[6]	valid_0's l2: 367.477
[7]	valid_0's l2: 360.978
[8]	valid_0's l2: 354.712
[9]	valid_0's l2: 348.457
[10]	valid_0's l2: 342.335
[11]	valid_0's l2: 336.34
[12]	valid_0's l2: 330.448
[13]	valid_0's l2: 324.671
[14]	valid_0's l2: 319.014
[15]	valid_0's l2: 313.448
[16]	valid_0's l2: 309.144
[17]	valid_0's l2: 303.781
[18]	valid_0's l2: 298.51
[19]	valid_0's l2: 293.357
[20]	valid_0's l2: 288.309
[21]	valid_0's l2: 283.361
[22]	valid_0's l2: 278.507
[23]	valid_0's l2: 273.738
[24]	valid_0's l2: 269.069
[25]	valid_0's l2: 264.48
[26]	valid_0's l2: 260
[27]	valid_0's l2: 255.604
[28]	valid_0's l2: 251.287
[29]	valid_0's l2: 247.063
[30]	valid_0's l2: 242.92
[31]	valid_0's l2: 238.948
[32]	valid_0's l2: 234.959
[33]	valid_0's l2: 231.036
[34]	valid_0's l2: 227.204
[35]	valid_0's l2: 223.512
[36]	valid_0's l2: 219.834
[37]	valid_0's l2: 216.454


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


96.02412959398825
[1]	valid_0's l2: 409.336
[2]	valid_0's l2: 402.286
[3]	valid_0's l2: 395.078
[4]	valid_0's l2: 388.008
[5]	valid_0's l2: 381.071
[6]	valid_0's l2: 374.411
[7]	valid_0's l2: 367.758
[8]	valid_0's l2: 361.335
[9]	valid_0's l2: 354.93
[10]	valid_0's l2: 348.661
[11]	valid_0's l2: 342.516
[12]	valid_0's l2: 336.494
[13]	valid_0's l2: 330.584
[14]	valid_0's l2: 324.79
[15]	valid_0's l2: 319.105
[16]	valid_0's l2: 314.71
[17]	valid_0's l2: 309.226
[18]	valid_0's l2: 303.831
[19]	valid_0's l2: 298.561
[20]	valid_0's l2: 293.392
[21]	valid_0's l2: 288.32
[22]	valid_0's l2: 283.342
[23]	valid_0's l2: 278.466
[24]	valid_0's l2: 273.691
[25]	valid_0's l2: 268.994
[26]	valid_0's l2: 264.411
[27]	valid_0's l2: 259.909
[28]	valid_0's l2: 255.497
[29]	valid_0's l2: 251.173
[30]	valid_0's l2: 246.93
[31]	valid_0's l2: 242.866
[32]	valid_0's l2: 238.786
[33]	valid_0's l2: 234.777
[34]	valid_0's l2: 230.86
[35]	valid_0's l2: 227.077
[36]	valid_0's l2: 223.316
[37]	valid_0's l2: 219.85

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


96.01630096296635


NameError: name 'k_fold' is not defined

In [12]:
ygq=test[['match_id','team_id','pred_score']]

test=pd.read_csv(path+'pubg_test.csv.zip',usecols=['match_id','team_id'])
test=pd.merge(test,ygq,on=['match_id','team_id'],how='left')

ygq=test['pred_score'].tolist()
pd.DataFrame({
    'team_placement': ygq
}).to_csv('submission.csv', index=None)

!zip submission.zip submission.csv

updating: submission.csv (deflated 72%)


In [ ]:
imp_df = pd.DataFrame()
imp_df['feats'] = lgb_Ranker.feature_name_
imp_df['imp'] = lgb_Ranker.feature_importances_
imp_df.sort_values('imp', ascending=False, inplace=True)
imp_df.to_csv('./imp_df1.csv')